# Part 0 & 1

# Load dataset

In [1]:
from IPython import embed
from datasets import load_dataset
from keras.src.layers import average

dataset = load_dataset('rotten_tomatoes')
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

C:\Users\Aozy\anaconda3\envs\sc4002\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df = train_dataset.to_pandas()
train_df.head()

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


# Create vocab from training dataset

In [3]:
import nltk


def build_vocab(train_dataset):
    # Create set, unique words only
    vocab = set()
    train_dataset_pos = []
    
    # Loop thru each sentence in training dataset
    for sentence in train_dataset['text']:
        # Basic text processing
        
        # Case folding
        sentence = sentence.lower()
        
        # NLTK tokenizer does a good job at separating meaningful words + punctuations
        # Better than defining regex ourselves
        word_list = nltk.tokenize.word_tokenize(sentence)
        
        # Further split words into separate words
        # e.g., 'well-being' -> 'well', 'being'
        # e.g., 'music/song' -> 'music', 'song'
        split_word_list = []
        for word in word_list:
            # If the word contains a hyphen or a slash, split it further
            if '-' in word or '/' in word:
                split_word_list.extend(word.replace('-', ' ').replace('/', ' ').split())
            else:
                split_word_list.append(word)
        
        # Dont remove all special characters, some are meaningful
        # Some words are surrounded by single/double quotes
        final_word_list = [word.strip("'\"") for word in split_word_list]
        
        # Add into set
        vocab.update(final_word_list)
        
        # Get pos tags
        # Also build POS tags
        pos_tags = nltk.pos_tag(final_word_list)
        train_dataset_pos.append(pos_tags)
        
    vocab.discard('')
    return vocab, train_dataset_pos

In [4]:
vocab, train_dataset_pos = build_vocab(train_dataset)

print(vocab)

{'gut', 'disconnects', 'soul', 'defining', 'pornography', 'objects', 'heft', 'caddyshack', 'scarcely', 'juggle', 'greedy', 'esoteric', 'perceptively', 'surrounded', 'space', 'conceit', 'extensively', 'pawn', 'lap', 'extemporaneously', 'imposed', 'famed', 'patronized', 'doubles', 'misdemeanor', 'traffics', 'vh1', 'schizophrenia', 'sneak', 'newsreels', 'enternecedora', 'cling', 'price', 'welty', 'simpsons', 'more', 'unlock', 'pesky', 'lee', 'destructive', 'wetsuit', 'albeit', 'verbal', 'tragically', 'reunions', 'hjelje', 'ish', 'stallion', 'jury', 'evaporation', 'infectious', 'ethos', 'handedness', 'soured', 'condone', 'fryers', 'refugee', 'curlers', 'drab', 'ineptly', 'você', 'hile', 'marrow', 'countless', 'atmosphere', 'reckless', 'tenor', 'luck', 'express', 'extensions', 'helped', 'twenty', 'opposed', 'prevents', 'forgiven', 'jackasses', 'intimidate', 'fixating', 'gawky', 'tighter', 'swimfan', 'government', 'institution', 'exposes', 'moron', 'fetishes', 'powered', 'member', 'sorimachi

In [5]:
print(train_dataset_pos[0])

[('the', 'DT'), ('rock', 'NN'), ('is', 'VBZ'), ('destined', 'VBN'), ('to', 'TO'), ('be', 'VB'), ('the', 'DT'), ('21st', 'JJ'), ('century', 'NN'), ('s', 'VBD'), ('new', 'JJ'), ('``', '``'), ('conan', 'JJ'), ('``', '``'), ('and', 'CC'), ('that', 'IN'), ('he', 'PRP'), ('s', 'VBZ'), ('going', 'VBG'), ('to', 'TO'), ('make', 'VB'), ('a', 'DT'), ('splash', 'NN'), ('even', 'RB'), ('greater', 'JJR'), ('than', 'IN'), ('arnold', 'RB'), ('schwarzenegger', 'NN'), (',', ','), ('jean', 'JJ'), ('claud', 'NN'), ('van', 'NN'), ('damme', 'NN'), ('or', 'CC'), ('steven', 'JJ'), ('segal', 'NN'), ('.', '.')]


## (a) What is the size of the vocabulary formed from your training data?

In [6]:
print(f'Vocab size/Unique words: {len(vocab)}')

Vocab size/Unique words: 16535


# Part 1: Preparing Word Embeddings

## Download GloVe embeddings: https://nlp.stanford.edu/projects/glove/
- Uncased means all words are lowercase

In [7]:
# Load GloVe embeddings
import numpy as np

def load_glove_embeddings(path):
    glove_embeddings = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float64')
            glove_embeddings[word] = vector
            
    return glove_embeddings

In [8]:
glove_embeddings = load_glove_embeddings('glove.6B.50d.txt')

In [9]:
print(f'Glove embedding matrix for "the":\n {glove_embeddings["the"]}')

Glove embedding matrix for "the":
 [ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01]


In [10]:
# Print size of the matrix
print(f'Number of unique words in embedding matrix: {len(glove_embeddings)}')

print(f'Number of Dimension/Features of Glove embedding matrix: {glove_embeddings["the"].shape[0]}')

Number of unique words in embedding matrix: 400000
Number of Dimension/Features of Glove embedding matrix: 50


## Create embedding matrix

In [11]:
# Finalize vocab
vocab_word_to_index = {word: idx for idx, word in enumerate(vocab)}

In [12]:
vocab_word_to_index

{'gut': 0,
 'disconnects': 1,
 'soul': 2,
 'defining': 3,
 'pornography': 4,
 'objects': 5,
 'heft': 6,
 'caddyshack': 7,
 'scarcely': 8,
 'juggle': 9,
 'greedy': 10,
 'esoteric': 11,
 'perceptively': 12,
 'surrounded': 13,
 'space': 14,
 'conceit': 15,
 'extensively': 16,
 'pawn': 17,
 'lap': 18,
 'extemporaneously': 19,
 'imposed': 20,
 'famed': 21,
 'patronized': 22,
 'doubles': 23,
 'misdemeanor': 24,
 'traffics': 25,
 'vh1': 26,
 'schizophrenia': 27,
 'sneak': 28,
 'newsreels': 29,
 'enternecedora': 30,
 'cling': 31,
 'price': 32,
 'welty': 33,
 'simpsons': 34,
 'more': 35,
 'unlock': 36,
 'pesky': 37,
 'lee': 38,
 'destructive': 39,
 'wetsuit': 40,
 'albeit': 41,
 'verbal': 42,
 'tragically': 43,
 'reunions': 44,
 'hjelje': 45,
 'ish': 46,
 'stallion': 47,
 'jury': 48,
 'evaporation': 49,
 'infectious': 50,
 'ethos': 51,
 'handedness': 52,
 'soured': 53,
 'condone': 54,
 'fryers': 55,
 'refugee': 56,
 'curlers': 57,
 'drab': 58,
 'ineptly': 59,
 'você': 60,
 'hile': 61,
 'marrow'

In [13]:
def create_embedding_matrix(word_to_index, glove_embeddings):
    # Initialize embedding matrix with zeros
    # 50d
    embedding_matrix = np.zeros((len(vocab), 50), dtype='float64')
    
    # Loop thru each word in vocab
    for word, idx in word_to_index.items():
        # Check if word exists in glove embeddings
        if word in glove_embeddings:
            # Copy glove embedding to embedding matrix
            embedding_matrix[idx] = glove_embeddings[word]
            # If OOV, assign None first
            
    return embedding_matrix

In [14]:
embedding_matrix = create_embedding_matrix(vocab_word_to_index, glove_embeddings)

In [15]:
embedding_matrix

array([[ 0.64909  , -0.70926  , -0.50309  , ..., -0.30989  ,  0.18346  ,
         0.46127  ],
       [ 0.38006  , -0.46419  , -0.0027137, ...,  0.12409  , -0.45251  ,
        -0.41407  ],
       [-0.57206  ,  0.67179  , -0.50949  , ..., -1.452    , -0.70341  ,
        -0.12087  ],
       ...,
       [ 0.63591  ,  0.28142  ,  1.103    , ...,  1.5154   ,  0.81081  ,
         0.11142  ],
       [ 0.4666   , -0.98096  , -0.13954  , ..., -0.14854  , -0.68896  ,
         0.31412  ],
       [-0.35729  , -0.43297  , -0.31115  , ..., -0.50281  ,  0.15226  ,
         0.40453  ]])

## (b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [16]:
def get_oov_words(embedding_matrix, vocab_word_to_index):
    oov_words = []
    
    for word, idx in vocab_word_to_index.items():
        # Check if zero matrix
        if np.sum(embedding_matrix[idx]) == 0:
            oov_words.append(word)

    return oov_words

In [17]:
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

In [18]:
oov_words.sort()
oov_words[:200]

['28k',
 '5ths',
 '8217',
 'abandone',
 'aborbing',
 'absolutamente',
 'aburrido',
 'acabamos',
 'accomodates',
 'aceitou',
 'achival',
 'achronological',
 'acontecimentos',
 'actorish',
 'actory',
 'actuación',
 'actuada',
 'addessi',
 'adorability',
 'adventues',
 'affirmational',
 "ain't",
 'alientation',
 'allodi',
 'amoses',
 'amusedly',
 'andamento',
 'animé',
 'anteing',
 'apallingly',
 'apesar',
 'aproveitar',
 'aqueles',
 "aren't",
 'arriesgado',
 'artnering',
 'artsploitation',
 'artístico',
 'assistir',
 'atacar',
 'atacarse',
 'atreve',
 'auteil',
 'autocritique',
 'b+',
 'bazadona',
 'bergmanesque',
 'beseechingly',
 'bibbidy',
 'bierbichler',
 'birot',
 'bizzarre',
 'bjorkness',
 'blighter',
 'blutarsky',
 'bobbidi',
 'bondish',
 'bornin',
 'bottomlessly',
 'bruckheimeresque',
 'brûlée',
 "bull's",
 'burningly',
 'bustingly',
 'butterfingered',
 'cadness',
 "cam'ron",
 'camareras',
 'cannier',
 'captivatingly',
 'capturou',
 'carente',
 'cativante',
 'certamente',
 'chabr

In [19]:
print(f"Number of OOV words: {len(oov_words)}")

Number of OOV words: 604


In [20]:
train_dataset_pos

[[('the', 'DT'),
  ('rock', 'NN'),
  ('is', 'VBZ'),
  ('destined', 'VBN'),
  ('to', 'TO'),
  ('be', 'VB'),
  ('the', 'DT'),
  ('21st', 'JJ'),
  ('century', 'NN'),
  ('s', 'VBD'),
  ('new', 'JJ'),
  ('``', '``'),
  ('conan', 'JJ'),
  ('``', '``'),
  ('and', 'CC'),
  ('that', 'IN'),
  ('he', 'PRP'),
  ('s', 'VBZ'),
  ('going', 'VBG'),
  ('to', 'TO'),
  ('make', 'VB'),
  ('a', 'DT'),
  ('splash', 'NN'),
  ('even', 'RB'),
  ('greater', 'JJR'),
  ('than', 'IN'),
  ('arnold', 'RB'),
  ('schwarzenegger', 'NN'),
  (',', ','),
  ('jean', 'JJ'),
  ('claud', 'NN'),
  ('van', 'NN'),
  ('damme', 'NN'),
  ('or', 'CC'),
  ('steven', 'JJ'),
  ('segal', 'NN'),
  ('.', '.')],
 [('the', 'DT'),
  ('gorgeously', 'RB'),
  ('elaborate', 'JJ'),
  ('continuation', 'NN'),
  ('of', 'IN'),
  ('``', '``'),
  ('the', 'DT'),
  ('lord', 'NN'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('rings', 'NNS'),
  ('``', '``'),
  ('trilogy', 'NN'),
  ('is', 'VBZ'),
  ('so', 'RB'),
  ('huge', 'JJ'),
  ('that', 'IN'),
  ('a', 'DT'),
  

In [21]:
# Find POS tags for OOV words
oov_words_pos = {}
for oov_word in oov_words:
    # Find all possible POS for OOV word
    pos_tags = []
    for sentence_pos in train_dataset_pos:
        for word, pos in sentence_pos:
            if word == oov_word:
                pos_tags.append(pos)
    # # Assign
    oov_words_pos[oov_word] = pos_tags

In [22]:
oov_words_pos

{'28k': ['CD'],
 '5ths': ['CD'],
 '8217': ['CD', 'CD'],
 'abandone': ['NN'],
 'aborbing': ['VBG'],
 'absolutamente': ['NN'],
 'aburrido': ['NN'],
 'acabamos': ['NN'],
 'accomodates': ['VBZ'],
 'aceitou': ['NN'],
 'achival': ['JJ'],
 'achronological': ['NN', 'JJ'],
 'acontecimentos': ['NNS'],
 'actorish': ['JJ'],
 'actory': ['JJ'],
 'actuación': ['FW'],
 'actuada': ['NN', 'JJ'],
 'addessi': ['NN'],
 'adorability': ['NN'],
 'adventues': ['NNS'],
 'affirmational': ['JJ'],
 "ain't": ['IN'],
 'alientation': ['NN'],
 'allodi': ['JJ'],
 'amoses': ['NNS'],
 'amusedly': ['RB'],
 'andamento': ['NN'],
 'animé': ['JJ', 'NN'],
 'anteing': ['VBG'],
 'apallingly': ['RB'],
 'apesar': ['NN'],
 'aproveitar': ['NN'],
 'aqueles': ['NNS'],
 "aren't": ['JJ'],
 'arriesgado': ['FW'],
 'artnering': ['VBG'],
 'artsploitation': ['NN'],
 'artístico': ['JJ'],
 'assistir': ['IN'],
 'atacar': ['NN'],
 'atacarse': ['JJ'],
 'atreve': ['VBP'],
 'auteil': ['NN', 'JJ'],
 'autocritique': ['NN'],
 'b+': ['NN'],
 'bazadona'

## (c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.

## Pass 1: Use Stemming to match OOV words of different forms to the same word in GloVe

In [23]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

# Stem vocab
stemmed_glove_vocab = {ps.stem(word): word for word in glove_embeddings.keys()}

# Stem vocab and OOV words, find same word
def find_substitute_word_stem(oov_word, stemmed_glove_vocab):
    stemmed_oov_word = ps.stem(oov_word)
    if stemmed_oov_word in stemmed_glove_vocab.keys():
        return stemmed_glove_vocab[stemmed_oov_word]
    
    return None

In [24]:
# Copy substitute word embedding to OOV word
for oov_word in oov_words:
    best_substitute_word = find_substitute_word_stem(oov_word, stemmed_glove_vocab)
    if best_substitute_word:
        print(f'OOV word: {oov_word}, substitute word: {best_substitute_word}')
        embedding_matrix[vocab_word_to_index[oov_word]] = glove_embeddings[best_substitute_word]

OOV word: 5ths, substitute word: 5th
OOV word: abandone, substitute word: abandonned
OOV word: accomodates, substitute word: accomodative
OOV word: aceitou, substitute word: aceite
OOV word: adorability, substitute word: ador
OOV word: affirmational, substitute word: affirmance
OOV word: allodi, substitute word: allodial
OOV word: anteing, substitute word: anting
OOV word: aqueles, substitute word: aquel
OOV word: birot, substitute word: birote
OOV word: bjorkness, substitute word: bjorke
OOV word: butterfingered, substitute word: butterfingers
OOV word: cadness, substitute word: cads
OOV word: capturou, substitute word: captures
OOV word: cineasts, substitute word: cineaste
OOV word: colonics, substitute word: colonizations
OOV word: complejos, substitute word: complejo
OOV word: complementares, substitute word: complementarities
OOV word: contructed, substitute word: contruction
OOV word: convencional, substitute word: convencion
OOV word: daneses, substitute word: danese
OOV word: d

In [25]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 483


## Pass 2: Use Wordnet synonyms bank based on sentence POS tagging

In [26]:
oov_words_pos

{'28k': ['CD'],
 '5ths': ['CD'],
 '8217': ['CD', 'CD'],
 'abandone': ['NN'],
 'aborbing': ['VBG'],
 'absolutamente': ['NN'],
 'aburrido': ['NN'],
 'acabamos': ['NN'],
 'accomodates': ['VBZ'],
 'aceitou': ['NN'],
 'achival': ['JJ'],
 'achronological': ['NN', 'JJ'],
 'acontecimentos': ['NNS'],
 'actorish': ['JJ'],
 'actory': ['JJ'],
 'actuación': ['FW'],
 'actuada': ['NN', 'JJ'],
 'addessi': ['NN'],
 'adorability': ['NN'],
 'adventues': ['NNS'],
 'affirmational': ['JJ'],
 "ain't": ['IN'],
 'alientation': ['NN'],
 'allodi': ['JJ'],
 'amoses': ['NNS'],
 'amusedly': ['RB'],
 'andamento': ['NN'],
 'animé': ['JJ', 'NN'],
 'anteing': ['VBG'],
 'apallingly': ['RB'],
 'apesar': ['NN'],
 'aproveitar': ['NN'],
 'aqueles': ['NNS'],
 "aren't": ['JJ'],
 'arriesgado': ['FW'],
 'artnering': ['VBG'],
 'artsploitation': ['NN'],
 'artístico': ['JJ'],
 'assistir': ['IN'],
 'atacar': ['NN'],
 'atacarse': ['JJ'],
 'atreve': ['VBP'],
 'auteil': ['NN', 'JJ'],
 'autocritique': ['NN'],
 'b+': ['NN'],
 'bazadona'

In [27]:
from nltk.corpus import wordnet as wn

pos_mapping_dict = {'NN':wn.NOUN,
              'JJ':wn.ADJ,
              'VB':wn.VERB,
              'RB':wn.ADV,
              # NLTK does not have wn.ADV_SAT
              }

# Convert oov_words_pos to wordnet POS
def map_pos_list(pos_list, mapping_dict):
    mapped_pos = []
    for pos in pos_list:
        # Find the first matching POS tag in mapping_dict by first letter comparison
        matched_pos = next((mapping_dict[key] for key in mapping_dict if key[0] == pos[0]), None)
        mapped_pos.append(matched_pos)
    return mapped_pos

# Creating the pos list
oov_words_pos_wordnet = {}
for key, pos_list in oov_words_pos.items():
    mapped_pos_list = map_pos_list(pos_list, pos_mapping_dict)
    
    # Remove None from list
    cleaned_list = [pos for pos in mapped_pos_list if pos is not None]
    
    # If all nones, remove whole entry
    if cleaned_list:
        oov_words_pos_wordnet[key] = cleaned_list
    

# Print the new dictionary
for key, value in oov_words_pos_wordnet.items():
    print(f"{key}: {value}")

abandone: ['n']
aborbing: ['v']
absolutamente: ['n']
aburrido: ['n']
acabamos: ['n']
accomodates: ['v']
aceitou: ['n']
achival: ['a']
achronological: ['n', 'a']
acontecimentos: ['n']
actorish: ['a']
actory: ['a']
actuada: ['n', 'a']
addessi: ['n']
adorability: ['n']
adventues: ['n']
affirmational: ['a']
alientation: ['n']
allodi: ['a']
amoses: ['n']
amusedly: ['r']
andamento: ['n']
animé: ['a', 'n']
anteing: ['v']
apallingly: ['r']
apesar: ['n']
aproveitar: ['n']
aqueles: ['n']
aren't: ['a']
artnering: ['v']
artsploitation: ['n']
artístico: ['a']
atacar: ['n']
atacarse: ['a']
atreve: ['v']
auteil: ['n', 'a']
autocritique: ['n']
b+: ['n']
bazadona: ['n']
bergmanesque: ['a']
beseechingly: ['r']
bibbidy: ['n']
bierbichler: ['n']
birot: ['n', 'n', 'n', 'n', 'n', 'n']
bizzarre: ['a']
bjorkness: ['n']
blighter: ['n']
blutarsky: ['n']
bobbidi: ['n']
bondish: ['a']
bornin: ['n']
bottomlessly: ['r']
bruckheimeresque: ['a']
brûlée: ['n']
bull's: ['n']
burningly: ['r']
bustingly: ['r']
butterfing

In [28]:
def find_substitute_wordnet_synonym(oov_word, oov_words_pos_wordnet):
    # Find synonyms for OOV words
    # Retrieve POS tags list
    try:
        pos_list = oov_words_pos_wordnet[oov_word]
    except KeyError:
        return None
    
    # Find number of unique pos, except None
    unique_pos = list(set(pos_list))
    # sort by count
    unique_pos.sort(key=lambda x: pos_list.count(x), reverse=True)
    
    # Loop thru each unique pos, try to find synonyms
    for possible_pos_tag in unique_pos:
        # Get synonyms
        for synset in wn.synsets(oov_word, pos=possible_pos_tag):
            for lemma in synset.lemmas():
                if lemma.name() in glove_embeddings:
                    print(f'OOV word: {oov_word}, synonym: {lemma.name()}')
                    return glove_embeddings[lemma.name()]

In [29]:
for oov_word in oov_words:
    synonym_glove_embedding = find_substitute_wordnet_synonym(oov_word, oov_words_pos_wordnet)
    if synonym_glove_embedding is not None:
        embedding_matrix[vocab_word_to_index[oov_word]] = synonym_glove_embedding

OOV word: cannier, synonym: cagey
OOV word: hastier, synonym: hasty
OOV word: excrescence, synonym: bulge
OOV word: blighter, synonym: pest
OOV word: fuddled, synonym: befuddle
OOV word: stuffiest, synonym: airless
OOV word: perfervid, synonym: ardent
OOV word: tardier, synonym: belated
OOV word: juiceless, synonym: dry
OOV word: divertingly, synonym: amusingly
OOV word: beseechingly, synonym: pleadingly
OOV word: greasiest, synonym: greasy
OOV word: captivatingly, synonym: enchantingly
OOV word: sappier, synonym: cockamamie
OOV word: shmear, synonym: schmear
OOV word: corniest, synonym: corny


In [30]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 467


## Pass 3: Use Edit Distance to solve misspelled OOV words

In [31]:
from Levenshtein import distance as lev

# Find most similar word for OOV word
def find_substitute_word_edit_dist(oov_word, glove_embeddings):
    # Set to infinity first
    min_dist = float('inf')
    closest_word = None
    
    # Loop thru all words in glove embeddings
    for word in glove_embeddings:
        # Calculate edit distance
        dist = lev(oov_word, word)
        
        # Update if new minimum distance found
        if dist < min_dist:
            min_dist = dist
            closest_word = word
            
    return closest_word, min_dist

In [32]:
min_dist_thresh = 1

for oov_word in oov_words:
    best_substitute_word, min_dist = find_substitute_word_edit_dist(oov_word, glove_embeddings)
    if min_dist <= min_dist_thresh:
        print(f'OOV word: {oov_word}, substitute word: {best_substitute_word}, Distance: {min_dist}')
        # Copy substitute word embedding to OOV word
        embedding_matrix[vocab_word_to_index[oov_word]] = glove_embeddings[best_substitute_word]

OOV word: hjelje, substitute word: hjelle, Distance: 1
OOV word: sillified, substitute word: villified, Distance: 1
OOV word: animé, substitute word: anime, Distance: 1
OOV word: rápidamente, substitute word: rapidamente, Distance: 1
OOV word: enviará, substitute word: enviar, Distance: 1
OOV word: don't, substitute word: dont, Distance: 1
OOV word: wankery, substitute word: wanker, Distance: 1
OOV word: b+, substitute word: by, Distance: 1
OOV word: bobbidi, substitute word: bobbili, Distance: 1
OOV word: shayamalan, substitute word: shyamalan, Distance: 1
OOV word: paródia, substitute word: parodia, Distance: 1
OOV word: fascinantes, substitute word: fascinates, Distance: 1
OOV word: funcionar, substitute word: funciona, Distance: 1
OOV word: dridi, substitute word: didi, Distance: 1
OOV word: clericks, substitute word: clerics, Distance: 1
OOV word: às, substitute word: 's, Distance: 1
OOV word: toolbags, substitute word: toolbars, Distance: 1
OOV word: fantasti, substitute word: fa

# Remaining OOV words

In [33]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 256


## Pass 4: Split words into subwords and average their subword embeddings

In [34]:
# Move split point from left to right of the OOV word
def find_subword(oov_word, glove_embeddings):
    # Usually words dont go beyond 3 splits, but we only try 2 splits, 3 splits might lose too much meaning
    # Different meaning: slappingly -> slap ping ly
    for i in range(1, len(oov_word)):
        left = oov_word[:i]
        right = oov_word[i:]
        
        if left in glove_embeddings and right in glove_embeddings.keys() and wn.synsets(left) and wn.synsets(right):
            return left, right
        
    return None

In [35]:
for oov_word in oov_words:
    subwords_tup = find_subword(oov_word, glove_embeddings)
    if subwords_tup is not None:
        print(f'OOV word: {oov_word}, subwords: {subwords_tup}')
        embeddings = [glove_embeddings[subword] for subword in subwords_tup]
        avg_embeddings = np.mean(embeddings, axis=0)
        embedding_matrix[vocab_word_to_index[oov_word]] = avg_embeddings

OOV word: cipherlike, subwords: ('cipher', 'like')
OOV word: kidlets, subwords: ('kid', 'lets')
OOV word: humbuggery, subwords: ('hum', 'buggery')
OOV word: autocritique, subwords: ('auto', 'critique')
OOV word: pokepie, subwords: ('poke', 'pie')
OOV word: unentertaining, subwords: ('un', 'entertaining')
OOV word: psychodramatics, subwords: ('psycho', 'dramatics')
OOV word: overstylized, subwords: ('over', 'stylized')
OOV word: unreligious, subwords: ('un', 'religious')
OOV word: reconceptualize, subwords: ('re', 'conceptualize')
OOV word: nutjob, subwords: ('nut', 'job')
OOV word: mibii, subwords: ('mib', 'ii')
OOV word: unconned, subwords: ('un', 'conned')
OOV word: superlarge, subwords: ('super', 'large')
OOV word: guessable, subwords: ('guess', 'able')
OOV word: unlaughable, subwords: ('un', 'laughable')
OOV word: overplotted, subwords: ('over', 'plotted')
OOV word: indieflick, subwords: ('indie', 'flick')
OOV word: landbound, subwords: ('land', 'bound')
OOV word: squaddie, subword

# Remaining OOV words

In [36]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 204


In [37]:
oov_words

['enternecedora',
 'shimmeringly',
 'entretenida',
 's1m0ne',
 'chopsocky',
 'talancón',
 'bottomlessly',
 'dudsville',
 'unforgivingly',
 'responsável',
 'fuhgeddaboutit',
 'começamos',
 'entretiene',
 'desarrollarse',
 'schneidermeister',
 'kibbitzes',
 'movie\x97if',
 'splatterfests',
 'arriesgado',
 'mullinski',
 'prefeminist',
 'real\x97she',
 'watstein',
 'bergmanesque',
 'joylessly',
 'atacar',
 'shagster',
 'péssima',
 'contemplarse',
 'predecesora',
 'representando',
 'incoloro',
 'provocatuers',
 'enfrentados',
 'djeinaba',
 'heartwarmingly',
 'kaputschnik',
 'patriotero',
 'estupendamente',
 'retadora',
 'solondzian',
 'addessi',
 'explicados',
 'actuada',
 'naturedness',
 'higuchinsky',
 'bibbidy',
 'inducingly',
 'contrária',
 'intelectualmente',
 "stalk'n'slash",
 'shlockmeister',
 'mediocridade',
 'revigorates',
 'ninguém',
 'kahlories',
 'emocionalmente',
 'mouglalis',
 'copmovieland',
 'blutarsky',
 'profundamente',
 'duración',
 'sparklingly',
 'absolutamente',
 'stra

## Last pass: Replace OOV words with <UNK> token

In [38]:
# Get min max range of glove embeddings
all_embeddings = np.stack(list(glove_embeddings.values()))
min_val = np.min(all_embeddings)
max_val = np.max(all_embeddings)

print(f"min: {min_val}")
print(f"max: {max_val}")

mean_embedding = np.mean(all_embeddings)
std_embedding = np.std(all_embeddings)

print(f"Mean: {mean_embedding}, Standard Deviation: {std_embedding}")

min: -5.4593
max: 5.3101
Mean: 0.020940489508694315, Standard Deviation: 0.6441042976813115


In [39]:
unk_token = '<UNK>'
# Random embedding for <UNK> token
embedding_dim = 50
unk_embedding = np.random.uniform(-0.25, 0.25, embedding_dim)

# Assign <UNK> token embedding to OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)
for oov_word in oov_words:
    embedding_matrix[vocab_word_to_index[oov_word]] = unk_embedding
    
# Add <UNK> token to vocab and embedding matrix
vocab_word_to_index[unk_token] = len(vocab)
embedding_matrix = np.vstack([embedding_matrix, unk_embedding])

In [40]:
# embedding_matrix[17841]

## Alternative methods to test
- semantic similarity with cosine similarity
- use model to predict oov embeddings Mimick https://github.com/yuvalpinter/Mimick


# Finally, save embedding matrix and vocab_to_index mapping

In [41]:
import pickle


with open('embedding_matrix.pkl', 'wb') as f:
    pickle.dump(embedding_matrix, f)
    
with open('vocab_word_to_index.pkl', 'wb') as f:
    pickle.dump(vocab_word_to_index, f)